# Experiment 3: Poisoning Attacks in Federated Learning

* Poisoning of
    * experiment_id = 0; label flipping 5 -> 4,  poisoned_clients[0,10, 25, 50, 75, 100, 125, 150, 200] rounds 200

In [1]:
from federated_learning.utils import SHAPUtil, experiment_util
from federated_learning import ClientPlane, Configuration
from federated_learning.server import Server
from datetime import datetime
import random

In [2]:
class ObserverConfiguration():
    experiment_type = "shap_fl_poisoned"
    experiment_id = 0
    test = False
    dataset_type = "MNIST"
    
    # Client Configurations 
    client_name = "client"
    client_type = "client"
    
    # Server Configurations 
    server_name = "server"
    server_type = "server"
    server_id = 0

In [3]:
def set_rounds(rounds):
    client_plane.set_rounds(rounds)
    server.set_rounds(rounds)
    
def update_configs():
    client_plane.update_config(config, observer_config)
    server.update_config(config, observer_config)
    
def run_round(rounds):
    # Federated Learning Round 
    client_plane.update_clients(server.get_nn_parameters())
    selected_clients = server.select_clients()
    client_parameters = client_plane.train_selected_clients(selected_clients)
    server.aggregate_model(client_parameters)

def select_random_clean():
    idx = 0
    while idx in client_plane.poisoned_clients:
        idx = random.randint(0,config.NUMBER_OF_CLIENTS)
    return idx

def train_poisoned_client_only(rounds): 
    client_plane.clients[client_plane.poisoned_clients[0]].train(rounds)
    client_plane.clients[client_plane.poisoned_clients[0]].push_metrics()
    if rounds == 5: 
        print(client_plane.clients[client_plane.poisoned_clients[0]].train_dataloader.dataset.dataset.targets[client_plane.clients[client_plane.poisoned_clients[0]].poisoned_indices][0])
    
def train_clean_client_only(idx, rounds): 
    client_plane.clients[idx].train(rounds)
    client_plane.clients[idx].push_metrics()

In [4]:
config = Configuration()
config.FROM_LABEL = 5
config.TO_LABEL = 4
data = config.DATASET(config)
shap_util = SHAPUtil(data.test_dataloader)
observer_config = ObserverConfiguration()

MNIST training data loaded.
MNIST test data loaded.


In [5]:
server = Server(config, observer_config,data.train_dataloader, data.test_dataloader, shap_util)
client_plane = ClientPlane(config, observer_config, data, shap_util)

default model saved to:./temp/models
Create 200 clients with dataset of size 300


In [6]:
import numpy as np
for num_p_clients in [200]:
    client_plane.reset_default_client_nets()
    server.reset_to_default_net()
    client_plane.reset_poisoning_attack()
    config.POISONED_CLIENTS = num_p_clients
    experiment_util.update_configs(client_plane, server, config, observer_config)
    client_plane.poison_clients()
    for i in range(200):
        set_rounds(i+1)
        run_round(i+1)
        if (i+1)%10 == 0:
            server.test()
            server.push_metrics()
        print("Round {} finished".format(i+1))

Load default model successfully
20/200 clients cleaned
40/200 clients cleaned
60/200 clients cleaned
80/200 clients cleaned
100/200 clients cleaned
120/200 clients cleaned
140/200 clients cleaned
160/200 clients cleaned
180/200 clients cleaned
200/200 clients cleaned
Cleaning successfully
Poison 200/200 clients
Flip 100.0% of the 5 labels to 4
[124  75 178  21  97  17  70  98  53  27  80 138  94 175 116   3 167  46
  87 127 105 140 137 166  74   4 169   1 145 107  36 141  69 149 182 139
 181 113  37 161  26  34  10 192  95 132 117  71  45 103  89 184  92  64
 177  28  40   2  30 106 129  50  57 110 162  43 108  54 126 186  31  88
 198 154 197 195  93 158  66 109  16  48 191  85  14  23  76 134  77 199
 142 144  72 121  83 185  38 180  90 147  62  42 135 133 183  33   6 174
 101  96 148 171  19  52  99 112  24 151  18  56 150 173  84  49 165 143
 119 189  15  86   5  51 155  68  58  67 163 193 125 187 164 179  44 104
 170  73   9 153 130 123 102  41  20 156 188  22  11  78 190  32  25 1

Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.


Successfully pushed server data to victoria metrics
Round 10 finished
Round 11 finished
Round 12 finished
Round 13 finished
Round 14 finished
Round 15 finished
Round 16 finished
Round 17 finished
Round 18 finished
Round 19 finished

Test set: Average loss: 0.0009, Accuracy: 8260/10000 (83%)

Successfully pushed server data to victoria metrics
Round 20 finished
Round 21 finished
Round 22 finished
Round 23 finished
Round 24 finished
Round 25 finished
Round 26 finished
Round 27 finished
Round 28 finished
Round 29 finished

Test set: Average loss: 0.0009, Accuracy: 8463/10000 (85%)

Successfully pushed server data to victoria metrics
Round 30 finished
Round 31 finished
Round 32 finished
Round 33 finished
Round 34 finished
Round 35 finished
Round 36 finished
Round 37 finished
Round 38 finished
Round 39 finished

Test set: Average loss: 0.0009, Accuracy: 8582/10000 (86%)

Successfully pushed server data to victoria metrics
Round 40 finished
Round 41 finished
Round 42 finished
Round 43 finish

In [ ]:
import numpy as np
import torch
torch.sum(server.confusion_matrix)

In [ ]:
server.get_shap_values()